In [1]:
import tensorflow as tf
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

In [2]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, Flatten, Dense, Dropout, Activation
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [3]:
import cv2
from PIL import Image
import tensorflow as tf
from random import randrange
import pickle
import json
import tensorflow.keras as keras
import os
import sys
import time

In [4]:
def load_shuffled_data():
    """
    Load dataset from pickle file and split features and labels
    returns (X_train, X_test, y_train, y_test)
    """
    start_time = time.time()

    with open("sample_train_img.pkl", 'rb') as f_name:
        train_img = pickle.load(f_name)
        train_img /= 255.0

    with open("sample_train_label.pkl", 'rb') as f_name:
        train_label = pickle.load(f_name)
        train_label = tf.keras.utils.to_categorical(train_label.astype('int'))
        
    
    with open("sample_test_img.pkl", 'rb') as f_name:
        test_img = pickle.load(f_name)
        test_img /= 255.0

    with open("sample_test_label.pkl", 'rb') as f_name:
        test_label = pickle.load(f_name)
        test_label = tf.keras.utils.to_categorical(test_label.astype('int'))

    #Shuffling Images!

    permute_train = np.random.permutation(len(train_label))
    permute_test = np.random.permutation(len(test_label))

    train_img = train_img[permute_train]
    train_label = train_label[permute_train]

    test_img = train_img[permute_train]
    test_label = train_label[permute_train]

    return(train_img, test_img, train_label, test_label)

    

In [5]:
def CNN_model(image_height = 256, image_width = 256, image_channel=1, class_count = 2):
    
    model = Sequential()
    model.add(Conv2D(   filters = 64, 
                        kernel_size=(9,9), 
                        strides = 2, 
                        activation='relu', 
                        kernel_initializer='he_uniform', 
                        input_shape = (image_height, image_width, image_channel)))

    model.add(BatchNormalization())

    #Taking the maximum in that subregion using MaxPooling
    model.add(MaxPool2D(    pool_size = (3, 3),
                            strides = (2, 2)))
    #he_uniform: Draw samples from uniform distributions within range of limits created with units

    model.add(Conv2D(   filters = 256, 
                        kernel_size=(5,5), 
                        strides = 1, 
                        activation='relu', 
                        kernel_initializer='he_uniform'))

    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2,2)))


    model.add(Conv2D(   filters = 128, 
                        kernel_size=(3,3), 
                        strides = 1, 
                        activation='relu', 
                        kernel_initializer='he_uniform'))

    model.add(BatchNormalization())
    model.add(MaxPool2D(    pool_size=(2,2)))
    
    model.add(Flatten())
    model.add(Dense(    units = 4096, 
                        activation='relu',              
                        kernel_initializer='he_uniform'))
    model.add(Dropout(0.5))   

    model.add(Dense(    units = 4096, 
                        activation='relu', 
                        kernel_initializer='he_uniform'))
    model.add(Dropout(0.5))   

    model.add(Dense(    units = 2, 
                        activation='softmax'))

    #Compiling:
    opt = SGD(lr = 0.001, momentum = 0.9)
    model.compile(  optimizer=opt, 
                    loss='categorical_crossentropy',
                    metrics = ['accuracy'])

    return model

In [6]:
cnn_model = CNN_model()
train_x, test_x, train_y, test_y = load_shuffled_data()

In [ ]:
model_name = 'CNN_3Layer_v2_1.h5'

In [7]:
#Fit Model!
#Replace text_x, test_y with validation data collected!
es = EarlyStopping(monitor='val_loss', mode='min', verbose=2, patience=10)
mc = ModelCheckpoint(model_name, monitor='val_accuracy', mode='max', verbose=2, save_best_only=True)
history = cnn_model.fit(train_x, train_y, epochs = 10, batch_size = 32, validation_split = 0.1, verbose = 1, callbacks = [es, mc])

Train on 180 samples, validate on 180 samples
Epoch 1/10
160/180 [=========================>....] - ETA: 2s - loss: 2.0320 - accuracy: 0.5437
Epoch 00001: val_accuracy improved from -inf to 0.50000, saving model to best_model_trial_3.h5
180/180 [==============================] - 24s 135ms/sample - loss: 1.9149 - accuracy: 0.5611 - val_loss: 0.8424 - val_accuracy: 0.5000
Epoch 2/10
160/180 [=========================>....] - ETA: 1s - loss: 0.9258 - accuracy: 0.7875
Epoch 00002: val_accuracy improved from 0.50000 to 0.52778, saving model to best_model_trial_3.h5
180/180 [==============================] - 23s 128ms/sample - loss: 0.9719 - accuracy: 0.7833 - val_loss: 0.9750 - val_accuracy: 0.5278
Epoch 3/10
160/180 [=========================>....] - ETA: 1s - loss: 0.8684 - accuracy: 0.8062
Epoch 00003: val_accuracy did not improve from 0.52778
180/180 [==============================] - 21s 118ms/sample - loss: 0.8636 - accuracy: 0.8056 - val_loss: 1.9096 - val_accuracy: 0.5111
Epoch 4/10

In [8]:
#Evaluate Data
loss, acc = cnn_model.evaluate(test_x, test_y, verbose = 0)

In [9]:
loss

1.8044600711928473

In [10]:
print(acc)

0.53333336

In [11]:
#Way solver!
#Trail 1: 75% - 52 %
#Trail 2: 88.89% - 57%
#Trail 3" 83%  - 53%